In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from sklearn.metrics import r2_score
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
#from wordcloud import WordCloud
from tqdm import tqdm
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
#from imblearn.over_sampling import SMOTE
#import plotly.express as px

In [2]:
data_set=pd.read_csv("mbti_1.csv")

In [3]:
data_set.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [4]:
data_set.columns

Index(['type', 'posts'], dtype='object')

In [5]:
data_set['type']

0       INFJ
1       ENTP
2       INTP
3       INTJ
4       ENTJ
        ... 
8670    ISFP
8671    ENFP
8672    INTP
8673    INFP
8674    INFP
Name: type, Length: 8675, dtype: object

In [6]:
data_set.describe(include="O")

,type,posts
count,8675,8675
unique,16,8675
top,INFP,'Now I'm interested. But too lazy to go resear...
freq,1832,1


In [7]:
data_set.describe()

,type,posts
count,8675,8675
unique,16,8675
top,INFP,'Now I'm interested. But too lazy to go resear...
freq,1832,1


In [8]:
print(data_set.dropna(inplace=True))

None


In [9]:
data_set["type"].value_counts()

INFP    1832
INFJ    1470
INTP    1304
INTJ    1091
ENTP     685
ENFP     675
ISTP     337
ISFP     271
ENTJ     231
ISTJ     205
ENFJ     190
ISFJ     166
ESTP      89
ESFP      48
ESFJ      42
ESTJ      39
Name: type, dtype: int64

In [10]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8675 entries, 0 to 8674
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    8675 non-null   object
 1   posts   8675 non-null   object
dtypes: object(2)
memory usage: 203.3+ KB


In [11]:
data_set["posts"][0]

"'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments  https://www.youtube.com/watch?v=iz7lE1g4XM4  sportscenter not top ten plays  https://www.youtube.com/watch?v=uCdfze1etec  pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8   http://www.youtube.com/watch?v=u8ejam5DP3E  On repeat for most of today.|||May the PerC Experience immerse you.|||The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   http://vimeo.com/22842206|||Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as...|||84389  84390  http://wallpaperpassion.com/upload/23700/friendship-boy-and-girl-wallpaper.jpg  http://assets.dornob.com/wp-content/uploads/2010/04/round-home-design.jpg ...

In [12]:
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    return text

In [13]:
count=0
for i in range(8675):
    if data_set["posts"][i].count("|||")!= 49:
        print("this one",i,"====>",data_set["posts"][i].count("|||"))
        count+=1
print("done",count)

this one 11 ====> 42
this one 16 ====> 48
this one 23 ====> 27
this one 28 ====> 47
this one 54 ====> 41
this one 55 ====> 57
this one 62 ====> 29
this one 64 ====> 31
this one 69 ====> 48
this one 84 ====> 46
this one 85 ====> 34
this one 87 ====> 28
this one 92 ====> 33
this one 93 ====> 37
this one 110 ====> 46
this one 121 ====> 17
this one 130 ====> 41
this one 146 ====> 44
this one 147 ====> 26
this one 155 ====> 11
this one 157 ====> 22
this one 167 ====> 58
this one 172 ====> 44
this one 176 ====> 42
this one 186 ====> 33
this one 187 ====> 46
this one 200 ====> 52
this one 207 ====> 55
this one 219 ====> 35
this one 221 ====> 41
this one 235 ====> 45
this one 239 ====> 43
this one 257 ====> 45
this one 275 ====> 20
this one 293 ====> 62
this one 297 ====> 41
this one 327 ====> 27
this one 349 ====> 43
this one 355 ====> 41
this one 356 ====> 37
this one 374 ====> 32
this one 375 ====> 35
this one 379 ====> 9
this one 384 ====> 9
this one 387 ====> 57
this one 388 ====> 65
this

In [14]:
data_set["posts"][774]

"'Says you.  He's not cute - he's pure evil in miniature form.  When he takes over the world, we will be forced to have Mexican hat day every 5th of the month. Those not wearing a mexican hat...'"

In [15]:
data_set['clean_posts'] = data_set['posts'].apply(cleanText)

In [16]:
data_set['clean_posts'][0]


"'<URL> <URL> enfp and intj moments  <URL>  sportscenter not top ten plays  <URL>  pranks What has been the most life-changing experience in your life? <URL>   <URL>  On repeat for most of today. May the PerC Experience immerse you. The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace~   <URL> Hello ENFJ7. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as... 84389  84390  <URL>  <URL> ... Welcome and stuff. <URL>  Game. Set. Match. Prozac, wellbrutin, at least thirty minutes of moving your legs (and I don't mean moving them while sitting in your same desk chair), weed in moderation (maybe try edibles as a healthier alternative... Basically come up with three items you've determined that each type (or whichever types you want to do) would more than likely use, given each types' cognitive functions and

In [17]:
train_data,test_data=train_test_split(data_set,test_size=0.2,random_state=42,stratify=data_set.type)

In [18]:
train_data

,type,posts,clean_posts
1228,INFP,'We are mandarin speakers. He receive educati...,'We are mandarin speakers. He receive educati...
1290,ISTP,"'Nope. Not now, not ever. I'm too busy with ...","'Nope. Not now, not ever. I'm too busy with ..."
6756,ENFJ,'That's the only one I haven't gotten to read ...,'That's the only one I haven't gotten to read ...
1662,INFP,'I used to think that maturity was burning bri...,'I used to think that maturity was burning bri...
3338,INFP,'I get typed as both a 4w5 and 5w6 as well but...,'I get typed as both a 4w5 and 5w6 as well but...
...,...,...,...
7292,INFP,Haven't posted here in a while. Here was my at...,Haven't posted here in a while. Here was my at...
1086,INFP,"'Ok, I'll go first. I'm a 29 year old INFP mal...","'Ok, I'll go first. I'm a 29 year old INFP mal..."
7435,ENTJ,"'I have dated a few INFJs, including my curren...","'I have dated a few INFJs, including my curren..."
1843,INTP,'People who are unable to replace social norms...,'People who are unable to replace social norms...


In [19]:
class Lemmatizer(object):
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, sentence):
        return [self.lemmatizer.lemmatize(word) for word in sentence.split() if len(word)>2]


In [20]:
vectorizer=TfidfVectorizer( max_features=5000,stop_words='english',tokenizer=Lemmatizer())
vectorizer.fit(train_data.posts)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


TfidfVectorizer(max_features=5000, stop_words='english',
                tokenizer=<__main__.Lemmatizer object at 0x000001B401A472B0>)

In [21]:
train_post=vectorizer.transform(train_data.posts).toarray()
test_post=vectorizer.transform(test_data.posts).toarray()

In [22]:
train_post.shape

(6940, 5000)

In [23]:
target_encoder=LabelEncoder()
train_target=target_encoder.fit_transform(train_data.type)
test_target=target_encoder.fit_transform(test_data.type)

In [ ]:
model_cat=CatBoostClassifier(loss_function='MultiClass',eval_metric='MultiClass',task_type='GPU',verbose=False)
model_cat.fit(train_post,train_target)
y=model_cat.predict(test_target)
